In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [ ]:
import sys
sys.path.insert(0, "/content/stylegan2")

In [ ]:
!git clone https://github.com/NVlabs/stylegan2.git

Cloning into 'stylegan2'...
remote: Enumerating objects: 93, done.
remote: Total 93 (delta 0), reused 0 (delta 0), pack-reused 93
Unpacking objects: 100% (93/93), done.


In [ ]:
!ls /content/stylegan2/

dataset_tool.py  LICENSE.txt		 README.md	   run_training.py
dnnlib		 metrics		 run_generator.py  test_nvcc.cu
Dockerfile	 pretrained_networks.py  run_metrics.py    training
docs		 projector.py		 run_projector.py


In [ ]:
!python /content/stylegan2/run_generator.py generate-images \
    --network=gdrive:networks/stylegan2-car-config-b.pkl \
  --seeds=6600-6625 --truncation-psi=0.5

In [ ]:
!mkdir out

mkdir: cannot create directory ‘out’: File exists


In [119]:
def seeder(seeds, vector_size):
  result = []

  for seed in seeds:
    rnd = np.random.RandomState(seed)
    result.append( rnd.randn(1, vector_size) ) 
  return result

In [103]:
# Copyright (c) 2019, NVIDIA Corporation. All rights reserved.
#
# This work is made available under the Nvidia Source Code License-NC.
# To view a copy of this license, visit
# https://nvlabs.github.io/stylegan2/license.html

import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys

import pretrained_networks

def generate_images(Gs, seeds, truncation_psi):
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() \
                  if name.startswith('noise')]

    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if truncation_psi is not None:
        Gs_kwargs.truncation_psi = truncation_psi

    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d/%d ...' % (seed_idx, len(seeds)))
        rnd = np.random.RandomState()
        tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(seed, None, **Gs_kwargs) # [minibatch, height, width, channel]
        path = f"/content/out/image{seed_idx}.png"
        PIL.Image.fromarray(images[0], 'RGB').save(path)




In [ ]:
sc = dnnlib.SubmitConfig()
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content"#/drive/My Drive/projects/stylegan2"
sc.run_desc = 'generate-images'
network_pkl = 'gdrive:networks/stylegan2-car-config-a.pkl'
print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
vector_size = Gs.input_shape[1:][0]
seeds = expand_seed( range(8000,8020), vector_size)




In [ ]:
generate_images(Gs, seeds,truncation_psi=0.5)

Generating image for seed 0/20 ...
Generating image for seed 1/20 ...
Generating image for seed 2/20 ...
Generating image for seed 3/20 ...
Generating image for seed 4/20 ...
Generating image for seed 5/20 ...
Generating image for seed 6/20 ...
Generating image for seed 7/20 ...
Generating image for seed 8/20 ...
Generating image for seed 9/20 ...
Generating image for seed 10/20 ...
Generating image for seed 11/20 ...
Generating image for seed 12/20 ...
Generating image for seed 13/20 ...
Generating image for seed 14/20 ...
Generating image for seed 15/20 ...
Generating image for seed 16/20 ...
Generating image for seed 17/20 ...
Generating image for seed 18/20 ...
Generating image for seed 19/20 ...


In [125]:
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content/out/stylegan2"
sc.run_desc = 'generate-images'
network_pkl = 'gdrive:networks/stylegan2-car-config-a.pkl'

print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
vector_size = Gs.input_shape[1:][0]
# range(8192,8300)
seeds = seeder( [11,875,4754,7770,8880], vector_size)
#generate_images(Gs, seeds,truncation_psi=0.5)
print(seeds[0].shape)

Loading networks from "gdrive:networks/stylegan2-car-config-a.pkl"...
(1, 512)


In [126]:
print(len(seeds))

5


In [128]:
# 8192+1,8192+9

STEPS = 500
diff = seeds[4] - seeds[0]
step = diff / STEPS
current = seeds[0].copy()

seeds2 = []
for i in range(STEPS):
  seeds2.append(current)
  current = current + step

generate_images(Gs, seeds2,truncation_psi=0.5)

Generating image for seed 0/500 ...
Generating image for seed 1/500 ...
Generating image for seed 2/500 ...
Generating image for seed 3/500 ...
Generating image for seed 4/500 ...
Generating image for seed 5/500 ...
Generating image for seed 6/500 ...
Generating image for seed 7/500 ...
Generating image for seed 8/500 ...
Generating image for seed 9/500 ...
Generating image for seed 10/500 ...
Generating image for seed 11/500 ...
Generating image for seed 12/500 ...
Generating image for seed 13/500 ...
Generating image for seed 14/500 ...
Generating image for seed 15/500 ...
Generating image for seed 16/500 ...
Generating image for seed 17/500 ...
Generating image for seed 18/500 ...
Generating image for seed 19/500 ...
Generating image for seed 20/500 ...
Generating image for seed 21/500 ...
Generating image for seed 22/500 ...
Generating image for seed 23/500 ...
Generating image for seed 24/500 ...
Generating image for seed 25/500 ...
Generating image for seed 26/500 ...
Generating 

In [108]:
%cd /content/out/

/content/out


In [109]:
!ls

image0.png    image281.png  image462.png  image643.png	image824.png
image100.png  image282.png  image463.png  image644.png	image825.png
image101.png  image283.png  image464.png  image645.png	image826.png
image102.png  image284.png  image465.png  image646.png	image827.png
image103.png  image285.png  image466.png  image647.png	image828.png
image104.png  image286.png  image467.png  image648.png	image829.png
image105.png  image287.png  image468.png  image649.png	image82.png
image106.png  image288.png  image469.png  image64.png	image830.png
image107.png  image289.png  image46.png   image650.png	image831.png
image108.png  image28.png   image470.png  image651.png	image832.png
image109.png  image290.png  image471.png  image652.png	image833.png
image10.png   image291.png  image472.png  image653.png	image834.png
image110.png  image292.png  image473.png  image654.png	image835.png
image111.png  image293.png  image474.png  image655.png	image836.png
image112.png  image294.png  image475.png  image656

In [131]:
!ffmpeg -r 30 -i image%d.png -vcodec mpeg4 -y vv2id.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [132]:
!cp /content/out/vv2id.mp4 /content/